## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [3]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        self.W1 = tf.Variable(tf.random.normal(shape=(28 * 28, 128), stddev=0.01), trainable=True)
        self.b1 = tf.Variable(tf.zeros(shape=(128,)), trainable=True)
        self.W2 = tf.Variable(tf.random.normal(shape=(128, 10), stddev=0.01), trainable=True)
        self.b2 = tf.Variable(tf.zeros(shape=(10,)), trainable=True)
        ####################
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x_flattened = tf.reshape(x, [-1, 784])
        h1 = tf.matmul(x_flattened, self.W1) + self.b1
        activated_h1 = tf.tanh(h1)
        logits = tf.matmul(activated_h1, self.W2) + self.b2
        return logits

model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [4]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [5]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer,
                                    tf.constant(train_data[0], dtype=tf.float32),
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model,
                      tf.constant(test_data[0], dtype=tf.float32),
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.301402 ; accuracy 0.09993333
epoch 1 : loss 2.301137 ; accuracy 0.10501666
epoch 2 : loss 2.3008716 ; accuracy 0.11038333
epoch 3 : loss 2.3006063 ; accuracy 0.116
epoch 4 : loss 2.300341 ; accuracy 0.12255
epoch 5 : loss 2.3000755 ; accuracy 0.1293
epoch 6 : loss 2.2998104 ; accuracy 0.13613333
epoch 7 : loss 2.2995446 ; accuracy 0.14241667
epoch 8 : loss 2.299279 ; accuracy 0.14933333
epoch 9 : loss 2.2990131 ; accuracy 0.15633333
epoch 10 : loss 2.298747 ; accuracy 0.16403334
epoch 11 : loss 2.2984807 ; accuracy 0.17183334
epoch 12 : loss 2.2982147 ; accuracy 0.18021667
epoch 13 : loss 2.297948 ; accuracy 0.18745
epoch 14 : loss 2.297681 ; accuracy 0.19461666
epoch 15 : loss 2.2974138 ; accuracy 0.20211667
epoch 16 : loss 2.2971463 ; accuracy 0.20976667
epoch 17 : loss 2.2968786 ; accuracy 0.21708333
epoch 18 : loss 2.2966106 ; accuracy 0.22471666
epoch 19 : loss 2.2963421 ; accuracy 0.23236667
epoch 20 : loss 2.2960734 ; accuracy 0.2394
epoch 21 : loss 2.2958043 ; 